In [1]:
model_path = "./fine_tuned_deberta_imdb"

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

c:\Users\Lenovo\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Lenovo\anaconda3\Lib\site-packages\transformers\convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [2]:
import pandas as pd
df = pd.read_csv("C:/Users/Lenovo/Desktop/NLP/Final_project/unseen_review/review_test_10.csv")

df.head(20)

,sentiment,review
0,Negative,I had no background knowledge of this movie be...
1,Negative,I am a huge Jane Austen fan and I ordered the ...
2,Negative,Nothing to say but Wow! Has anyone actually ha...
3,Negative,i like Jane Austin novels. I love Pride and Pr...
4,Negative,In this day and age of incredible special movi...
5,Negative,Becky Harris plays the female shopper whose mi...
6,Negative,Watching TRUTH ABOUT LOVE (is this a double en...
7,Negative,"A painfully protracted, maudlin and predictabl..."
8,Negative,"Let's be honest. As a film school project, mad..."
9,Negative,A terrible movie as everyone has said. What ma...


In [3]:
import pandas as pd
df = pd.read_csv("C:/Users/Lenovo/Desktop/NLP/Final_project/unseen_review/label_review_test_10.csv")

df.head(20)

,sentiment,review
0,Negative,I had no background knowledge of this movie be...
1,Negative,I am a huge Jane Austen fan and I ordered the ...
2,Negative,Nothing to say but Wow! Has anyone actually ha...
3,Negative,i like Jane Austin novels. I love Pride and Pr...
4,Negative,In this day and age of incredible special movi...
5,Positive,I would have rated the series a perfect 10 for...
6,Positive,Greetings again from the darkness. Insight int...
7,Positive,I went to a small advance screening of this mo...
8,Positive,'Holes' is my all-time favorite movie! So far ...
9,Positive,It really is a shame that films like this neve...


In [3]:
import pandas as pd
import torch 

# ✅ โหลด CSV
df = pd.read_csv("C:/Users/Lenovo/Desktop/NLP/Final_project/unseen_review/review_test_10.csv")

# ✅ tokenize ทั้งหมด
inputs = tokenizer(df["review"].tolist(), padding=True, truncation=True, return_tensors="pt")

# ✅ predict
outputs = model(**inputs)
probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
pred_labels = probs.argmax(axis=1)

# ✅ เพิ่มคอลัมน์ผลลัพธ์
df["predicted_sentiment"] = ["positive" if p == 1 else "negative" for p in pred_labels]

# ✅ ดูตัวอย่าง
print(df)

# ✅ ถ้าอยากบันทึกผล
#df.to_csv("reviews_with_predictions.csv", index=False)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


                                               review predicted_sentiment
0   I had no background knowledge of this movie be...            negative
1   I am a huge Jane Austen fan and I ordered the ...            negative
2   Nothing to say but Wow! Has anyone actually ha...            negative
3   i like Jane Austin novels. I love Pride and Pr...            negative
4   In this day and age of incredible special movi...            negative
5   Becky Harris plays the female shopper whose mi...            negative
6   Watching TRUTH ABOUT LOVE (is this a double en...            negative
7   A painfully protracted, maudlin and predictabl...            negative
8   Let's be honest. As a film school project, mad...            negative
9   A terrible movie as everyone has said. What ma...            negative
10  It's really annoying when good movies like thi...            positive
11  The daughter's words are poetry: 'I can't go o...            positive
12  The best Treasure Island ever made

In [ ]:
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, classification_report


#  โหลด CSV

df = pd.read_csv("C:/Users/Lenovo/Desktop/NLP/Final_project/unseen_review/label_review_test_10.csv")

#  tokenize ข้อความจาก column review
inputs = tokenizer(df["review"].tolist(), padding=True, truncation=True, return_tensors="pt")

#  predict
outputs = model(**inputs)
probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
pred_labels = probs.argmax(axis=1).numpy()

#  เพิ่มผลลัพธ์เข้า DataFrame
df["predicted_label"] = pred_labels
df["predicted_sentiment"] = ["positive" if p == 1 else "negative" for p in pred_labels]

#  แปลง label จริงเป็น 0/1
if df["sentiment"].dtype == object:  
    df["true_label"] = df["sentiment"].apply(lambda x: 1 if x.lower()=="positive" else 0)
else:
    df["true_label"] = df["sentiment"]


print("\n=== Sample Predictions ===")
sample_df = df.head(5) 
for i in range(len(sample_df)):
    text = sample_df.loc[i, "review"][:80]  # ตัดข้อความให้สั้นหน่อย
    true_label = sample_df.loc[i, "sentiment"]
    pred_label = sample_df.loc[i, "predicted_sentiment"]
    print(f"Review: {text}...")
    print(f" True: {true_label} | Predicted: {pred_label}\n")    

# ✅ คำนวณ Accuracy
acc = accuracy_score(df["true_label"], df["predicted_label"])
print(f"✅ Accuracy: {acc*100:.2f}%")

# ✅ รายงาน Precision / Recall / F1
print(classification_report(df["true_label"], df["predicted_label"], target_names=["negative", "positive"]))

# ✅ บันทึกผลลัพธ์ CSV ใหม่
#df.to_csv("reviews_with_predictions.csv", index=False)
#print("✅ บันทึกไฟล์ reviews_with_predictions.csv สำเร็จ!")




=== Sample Predictions ===
Review: I had no background knowledge of this movie before I bought it, but it sounded c...
 True: Negative | Predicted: negative

Review: I am a huge Jane Austen fan and I ordered the movie from Amazon.UK just so I cou...
 True: Negative | Predicted: negative

Review: Nothing to say but Wow! Has anyone actually had somebody sneak up on them in an ...
 True: Negative | Predicted: negative

Review: i like Jane Austin novels. I love Pride and Prejudice and Sense and Sensibility ...
 True: Negative | Predicted: negative

Review: In this day and age of incredible special movie effects, this one was a sore dis...
 True: Negative | Predicted: negative

✅ Accuracy: 100.00%
              precision    recall  f1-score   support

    negative       1.00      1.00      1.00         5
    positive       1.00      1.00      1.00         5

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00  

In [2]:
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm  # progress bar

BATCH_SIZE = 32  #  batch_size

# load CSV
df = pd.read_csv("C:/Users/Lenovo/Desktop/NLP/Final_project/unseen_review/review_test_10.csv")

all_pred_labels = []

# loop batch
for i in tqdm(range(0, len(df), BATCH_SIZE)):
    batch_texts = df["review"].iloc[i:i+BATCH_SIZE].tolist()
    
    # tokenize batch
    inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")
    
    # predict
    with torch.no_grad():  
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        batch_preds = probs.argmax(axis=1).cpu().numpy()
    
    all_pred_labels.extend(batch_preds)

#  DataFrame
df["predicted_label"] = all_pred_labels
df["predicted_sentiment"] = ["positive" if p == 1 else "negative" for p in all_pred_labels]

# convert label 0/1
if df["sentiment"].dtype == object:  
    df["true_label"] = df["sentiment"].apply(lambda x: 1 if x.lower()=="positive" else 0)
else:
    df["true_label"] = df["sentiment"]

# cal Accuracy
acc = accuracy_score(df["true_label"], df["predicted_label"])
print(f"\n✅ Accuracy: {acc*100:.2f}%")

# Precision / Recall / F1
print("\n=== Classification Report ===")
print(classification_report(df["true_label"], df["predicted_label"], target_names=["negative", "positive"]))

# show 10 row head
print("\n=== Sample Predictions (first 10) ===")
sample_df = df.head(10)
for _, row in sample_df.iterrows():
    print(f"Review: {row['review'][:80]}...")
    print(f"  ✅ True: {row['sentiment']} | 🔮 Predicted: {row['predicted_sentiment']}\n")


# save CSV file
# C:/Users/Lenovo/Desktop/NLP/Final_project/prediction_result/reviews_with_predictions_Deberta_model.csv

df.to_csv("C:/Users/Lenovo/Desktop/NLP/Final_project/prediction_result/reviews_with_predictions_Deberta_model.csv", index=False)
print(" save file reviews_with_predictions_Deberta_model.csv ")


  0%|          | 0/6 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


100%|██████████| 6/6 [10:05<00:00, 100.99s/it]



✅ Accuracy: 97.74%

=== Classification Report ===
              precision    recall  f1-score   support

    negative       0.97      0.99      0.98        99
    positive       0.99      0.96      0.97        78

    accuracy                           0.98       177
   macro avg       0.98      0.98      0.98       177
weighted avg       0.98      0.98      0.98       177


=== Sample Predictions (first 10) ===
Review: I had no background knowledge of this movie before I bought it, but it sounded c...
  ✅ True: Negative | 🔮 Predicted: negative

Review: I am a huge Jane Austen fan and I ordered the movie from Amazon.UK just so I cou...
  ✅ True: Negative | 🔮 Predicted: negative

Review: Nothing to say but Wow! Has anyone actually had somebody sneak up on them in an ...
  ✅ True: Negative | 🔮 Predicted: negative

Review: i like Jane Austin novels. I love Pride and Prejudice and Sense and Sensibility ...
  ✅ True: Negative | 🔮 Predicted: negative

Review: In this day and age of incredib

In [ ]:
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm  # progress bar

BATCH_SIZE = 16  # ✅ ปรับตาม VRAM

# ✅ เลือกอุปกรณ์ (GPU ถ้ามี, ถ้าไม่มีใช้ CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# ✅ ย้ายโมเดลไป GPU
model = model.to(device)

# ✅ โหลด CSV
df = pd.read_csv("C:/Users/Lenovo/Desktop/NLP/Final_project/unseen_review/review_test_10.csv")


all_pred_labels = []

# ✅ predict แบบ batch
for i in tqdm(range(0, len(df), BATCH_SIZE)):
    batch_texts = df["review"].iloc[i:i+BATCH_SIZE].tolist()
    
    # ✅ tokenize + ส่งไป GPU
    inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to(device)
    
    # ✅ predict
    with torch.no_grad():  
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        batch_preds = probs.argmax(axis=1).cpu().numpy()  # ✅ กลับ CPU เพื่อเก็บ
    
    all_pred_labels.extend(batch_preds)

# ✅ เพิ่มผลลัพธ์เข้า DataFrame
df["predicted_label"] = all_pred_labels
df["predicted_sentiment"] = ["positive" if p == 1 else "negative" for p in all_pred_labels]

# ✅ แปลง label จริงเป็น 0/1
if "sentiment" in df.columns:  # ถ้ามี label จริง
    if df["sentiment"].dtype == object:  
        df["true_label"] = df["sentiment"].apply(lambda x: 1 if x.lower()=="positive" else 0)
    else:
        df["true_label"] = df["sentiment"]

    # ✅ คำนวณ Accuracy
    acc = accuracy_score(df["true_label"], df["predicted_label"])
    print(f"\n Accuracy: {acc*100:.2f}%")

    # ✅ Precision / Recall / F1
    print("\n=== Classification Report ===")
    print(classification_report(df["true_label"], df["predicted_label"], target_names=["negative", "positive"]))
else:
    print("\n No ground-truth labels found, skipping accuracy calculation")

# ✅ แสดงตัวอย่าง 10 row แรก
print("\n=== Sample Predictions (first 10) ===")
for _, row in df.head(10).iterrows():
    print(f"Review: {row['review'][:80]}...")
    if "sentiment" in df.columns:
        print(f"  ✅ True: {row['sentiment']} | 🔮 Predicted: {row['predicted_sentiment']}\n")
    else:
        print(f"  🔮 Predicted: {row['predicted_sentiment']}\n")

# ✅ บันทึกผลลัพธ์
df.to_csv("C:/Users/Lenovo/Desktop/NLP/Final_project/prediction_result/reviews_with_predictions_Deberta_model.csv", index=False)
print("\n✅ Saved predictions to reviews_with_predictions_Deberta_model.csv")


✅ Using device: cuda


100%|██████████| 12/12 [01:22<00:00,  6.88s/it]


 Accuracy: 97.74%

=== Classification Report ===
              precision    recall  f1-score   support

    negative       0.97      0.99      0.98        99
    positive       0.99      0.96      0.97        78

    accuracy                           0.98       177
   macro avg       0.98      0.98      0.98       177
weighted avg       0.98      0.98      0.98       177


=== Sample Predictions (first 10) ===
Review: I had no background knowledge of this movie before I bought it, but it sounded c...
  ✅ True: Negative | 🔮 Predicted: negative

Review: I am a huge Jane Austen fan and I ordered the movie from Amazon.UK just so I cou...
  ✅ True: Negative | 🔮 Predicted: negative

Review: Nothing to say but Wow! Has anyone actually had somebody sneak up on them in an ...
  ✅ True: Negative | 🔮 Predicted: negative

Review: i like Jane Austin novels. I love Pride and Prejudice and Sense and Sensibility ...
  ✅ True: Negative | 🔮 Predicted: negative

Review: In this day and age of incredibl

In [2]:
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm  # progress bar

BATCH_SIZE = 8  # ✅ ปรับตาม VRAM

# ✅ เลือกอุปกรณ์ (GPU ถ้ามี, ถ้าไม่มีใช้ CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# ✅ ย้ายโมเดลไป GPU
model = model.to(device)

# ✅ โหลด CSV
df = pd.read_csv("C:/Users/Lenovo/Desktop/NLP/Final_project/unseen_review/review.csv")


all_pred_labels = []

# ✅ predict แบบ batch
for i in tqdm(range(0, len(df), BATCH_SIZE)):
    batch_texts = df["review"].iloc[i:i+BATCH_SIZE].tolist()
    
    # ✅ tokenize + ส่งไป GPU
    inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to(device)
    
    # ✅ predict
    with torch.no_grad():  
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        batch_preds = probs.argmax(axis=1).cpu().numpy()  # ✅ กลับ CPU เพื่อเก็บ
    
    all_pred_labels.extend(batch_preds)

# ✅ เพิ่มผลลัพธ์เข้า DataFrame
df["predicted_label"] = all_pred_labels
df["predicted_sentiment"] = ["positive" if p == 1 else "negative" for p in all_pred_labels]

# ✅ แปลง label จริงเป็น 0/1
if "sentiment" in df.columns:  # ถ้ามี label จริง
    if df["sentiment"].dtype == object:  
        df["true_label"] = df["sentiment"].apply(lambda x: 1 if x.lower()=="positive" else 0)
    else:
        df["true_label"] = df["sentiment"]

    # ✅ คำนวณ Accuracy
    acc = accuracy_score(df["true_label"], df["predicted_label"])
    print(f"\n Accuracy: {acc*100:.2f}%")

    # ✅ Precision / Recall / F1
    print("\n=== Classification Report ===")
    print(classification_report(df["true_label"], df["predicted_label"], target_names=["negative", "positive"]))
else:
    print("\n No ground-truth labels found, skipping accuracy calculation")

# ✅ แสดงตัวอย่าง 10 row แรก
print("\n=== Sample Predictions (first 10) ===")
for _, row in df.head(10).iterrows():
    print(f"Review: {row['review'][:80]}...")
    if "sentiment" in df.columns:
        print(f"  ✅ True: {row['sentiment']} | 🔮 Predicted: {row['predicted_sentiment']}\n")
    else:
        print(f"  🔮 Predicted: {row['predicted_sentiment']}\n")

# ✅ บันทึกผลลัพธ์
df.to_csv("C:/Users/Lenovo/Desktop/NLP/Final_project/prediction_result/reviews_with_predictions_Deberta_model_full.csv", index=False)
print("\n✅ Saved predictions to reviews_with_predictions_Deberta_model_full.csv")


✅ Using device: cuda


100%|██████████| 1250/1250 [1:18:51<00:00,  3.79s/it]



 Accuracy: 95.92%

=== Classification Report ===
              precision    recall  f1-score   support

    negative       0.96      0.96      0.96      5081
    positive       0.96      0.96      0.96      4919

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000


=== Sample Predictions (first 10) ===
Review: I had no background knowledge of this movie before I bought it, but it sounded c...
  ✅ True: Negative | 🔮 Predicted: negative

Review: I am a huge Jane Austen fan and I ordered the movie from Amazon.UK just so I cou...
  ✅ True: Negative | 🔮 Predicted: negative

Review: Nothing to say but Wow! Has anyone actually had somebody sneak up on them in an ...
  ✅ True: Negative | 🔮 Predicted: negative

Review: i like Jane Austin novels. I love Pride and Prejudice and Sense and Sensibility ...
  ✅ True: Negative | 🔮 Predicted: negative

Review: In this day and age of incredibl